## Installation and deployment of 5G System using the Open Air Interface (OAI)

In this tutorial we describe how to configure and run a 5G end-to-end setup with OAI CN5G, OAI gNB and OAI nrUE.
Minimum hardware requirements:

- Laptop/Desktop/Server for OAI CN5G and OAI gNB
    - Operating System: Ubuntu 22.04 LTS
    - CPU: 8 cores x86_64 @ 3.5 GHz
    - RAM: 32 GB
- Laptop for UE
    - Operating System: Ubuntu 22.04 LTS
    - CPU: 8 cores x86_64 @ 3.5 GHz
    - RAM: 8 GB


## Objectives
- Install and Deploy CN5G, OAIgNB and nrUE using OAI
    - CN5G version 2.0.1
    - OAIgNB and nrUE version 2.1.0

## CN5G Installation

In a Linux Terminal, follow the steps below:

**Step 0:** Setup your VM or Baremetal Machine, after install Useful Software

**Step 1:** Install pre-requisites

Open a terminal in your home directory and then execute the following commands:
```
sudo su
```
```
apt update -y
```
```
apt upgrade -y
```
```
apt install -y python3-pip git curl cmake tree golang vim net-tools build-essential nfs-common openssh-server putty
```
```
sudo apt install -y ca-certificates curl gnupg
```
```
sudo install -m 0755 -d /etc/apt/keyrings
```
```
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /etc/apt/keyrings/docker.gpg
```
```
sudo chmod a+r /etc/apt/keyrings/docker.gpg
```
```
echo "deb [arch="$(dpkg --print-architecture)" signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu "$(. /etc/os-release && echo "$VERSION_CODENAME")" stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
```
```
sudo apt update
```
```
sudo apt install -y docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin
```
```
sudo curl -L https://github.com/docker/compose/releases/download/2.26.0/docker-compose-`uname -s`-`uname -m` -o /usr/local/bin/docker-compose
```
```
sudo usermod -a -G docker $(whoami)
```

**Step 2:** Reboot your operational system
```
reboot
```
**Step 3:** Download CN5G files

Create a folder called OAI in your home directory:
```
mkdir ~/OAI
```
```
cd OAI
```
```
git clone https://gitlab.eurecom.fr/oai/cn5g/oai-cn5g-fed.git
```
```
cd oai-cn5g-fed
```
```
git checkout -f v2.0.1
```
**Step 4:** Pull the images from Docker Hub

In the oai-cn5g-fed directory:
```
cd docker-compose
```
```
docker compose -f docker-compose-basic-nrf.yaml pull
```
You will see:

![5GC_deploy_02](../FIGS/H01_5G_Complete_Deployment/img_01.png)

**Step 5:** Start OAI CN5G
In the CN5G terminal, type:
```
docker compose -f docker-compose-basic-nrf.yaml up -d
```
You will see:

![5GC_deploy_02](../FIGS/H01_5G_Complete_Deployment/img_02.png)

**Step 6:** See all CN5G docker containers
In the CN5G terminal, type:
```
docker ps -a
```
The expected visualization is similar to the figure below:

![5GC_deploy_02](../FIGS/H01_5G_Complete_Deployment/img_03.png)

**Step 7:** See logs of the 5GC's AMF container

In the CN5G terminal, type:
```
docker logs -f oai-amf
```
The expected visualization is similar to the figure below:

![5GC_deploy_02](../FIGS/H01_5G_Complete_Deployment/img_04.png)

You can also use 
```
docker logs -f ff850c663eea
```
where ff850c663eea is the id of the container in your machine.

To leave this visualization press Ctrl + c.

**Step 8:** Stop OAI 5GC

In the CN5G terminal, type:
```
docker compose -f docker-compose-basic-nrf.yaml down
```
You will see:

![5GC_deploy_02](../FIGS/H01_5G_Complete_Deployment/img_05.png)

## OAIgNB and nrUE Installation

P.S.: From this moment on, some command lines and directory adresses will have "YOUR_USER" in their composition, remember to change it to the user of your virtual machine.

**Step 1:** Install pre-requisites

Open a terminal in your home directory, then execute the following commands:
```
sudo apt install -y autoconf automake build-essential ccache cmake cpufrequtils doxygen ethtool g++ git inetutils-tools libboost-all-dev libncurses5 libncurses5-dev libusb-1.0-0 libusb-1.0-0-dev libusb-dev python3-dev python3-mako python3-numpy python3-requests python3-scipy python3-setuptools python3-ruamel.yaml
```
```
cd OAI
```
```
git clone https://github.com/EttusResearch/uhd.git
```
```
cd uhd
```
```
git checkout v4.6.0.0
```
```
cd host
```
```
mkdir build
```
```
cd build
```
```
cmake ../
```
```
make -j $(nproc)
```
```
make test
```
``` 
sudo make install
```
```
sudo ldconfig
```
```
sudo uhd_images_downloader
```

**Step 2:** Download OAIgNB and nrUE files

Open a terminal in OAI directory, then execute the following command lines:
```
git clone https://gitlab.eurecom.fr/oai/openairinterface5g.git
```
```
cd openairinterface5g
```
```
git checkout v2.1.0
```

**Step 3:** Build the configuration files

Open a terminal in the openairinterface5g repository:
```
cd cmake_targets
```
```
./build_oai -I
```
```
sudo apt install -y libforms-dev libforms-bin
```
```
./build_oai -w USRP --ninja --nrUE --gNB --build-lib "nrscope" -C
```

**Attention:** Be patient, the last command could take a whileeeee. If you want to follow the installation logs, run the command below:
```
tail -f /home/YOUR_USER/OAI/openairinterface5g/cmake_targets/log/all.txt
```

## Change Values in the Configuration Files

It's necessary to change some configuration values in diferent files, so, let's start with the gNB.

### OAIgNB Configuration for the Monolithic Mode

**Step 1:** Configure the PLMN and TAC values

The Public Land Mobile Network (PLMN) and Tracking Area Code (TAC) will be configured in the monolithic RAN (/home/YOUR_USER/OAI/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf) configuration file.

Figure below shows the part of the code that must be modified in the monolithic file (lines 13 and 14). The parameters shown in figure should be modified according to the values present in docker compose respective file, in the "oai-amf" session (/home/YOUR_USER/OAI/oai-cn5g-fed/docker-compose/conf/basic_nrf_config.yaml).

![5GC_deploy_02](../FIGS/H01_5G_Complete_Deployment/img_06.png)

Figure below shows the parameters of the plmn session that should be checked and passed to the monolithic configuration file: tac (referring to "tracking_area_code"), mcc (referring to " plmn_list mcc") and PLMN_SUPPORT_MNC (referring to " plmn_listmnc").

![5GC_deploy_02](../FIGS/H01_5G_Complete_Deployment/img_07.png)

Figure below shows the result in the monolithic RAN file (/home/YOUR_USER/OAI/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf).

![5GC_deploy_02](../FIGS/H01_5G_Complete_Deployment/img_08.png)

### nrE Configuration

**Step 1:** Configure the IMSI, Permanent Key and OPC

The UE parameters file is /home/YOUR_USER/OAI/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf. The first three lines of the file are composed by the IMSI, Permanent Key and OPC, as shown in figure below. You will configure these parameters.

![5GC_deploy_02](../FIGS/H01_5G_Complete_Deployment/img_09.png)

You need to check in the docker-compose file (/home/YOUR_USER/OAI/oai-cn5g-fed/docker-compose/docker-compose-basic-nrf.yaml) which SQL file the core is using. See the figure below.

![5GC_deploy_02](../FIGS/H01_5G_Complete_Deployment/img_10.png)

Verify/change the parameters in UE file (/home/YOUR_USER/OAI/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf) according to the SQL file illustrated in figure below. The parameters are:

- "ueid" refers to "imsi"
- "encPermanentKey" refers to "key"
- "encOpcKey" refers to "opc

![5GC_deploy_02](../FIGS/H01_5G_Complete_Deployment/img_11.png)

Figure below shows the result in the UE configuration file (/home/YOUR_USER/OAI/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf).

![5GC_deploy_02](../FIGS/H01_5G_Complete_Deployment/img_12.png)

## System Simulation

Open 3 others terminals in the your home repository. The arrangement for the following steps can follow that shown in the figure below:

![5GC_deploy_02](../FIGS/H01_5G_Complete_Deployment/img_13.png)

In the first terminal, execute the commands below:
```
cd ~/OAI/oai-cn5g-fed/docker-compose
```
```
docker compose -f docker-compose-basic-nrf.yaml up -d
```
In the second one follow the commands below:
```
cd ~/OAI/openairinterface5g/cmake_targets/ran_build/build
```
```
sudo ./nr-softmodem -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf --gNBs.[0].min_rxtxtime 6 --rfsim --sa
```
In the third, follow the commands below:
```
cd ~/OAI/openairinterface5g/cmake_targets/ran_build/build
```
```
sudo ./nr-uesoftmodem -r 106 --numerology 1 --band 78 -C 3619200000 --sa --uicc0.imsi 001010000000001 --rfsim
```

**Step 2:** See logs of the 5GC's AMF container

In the CN5G terminal, type:
```
docker logs -f oai-amf
```
The expected visualization is similar to the figure below:

![5GC_deploy_02](../FIGS/H01_5G_Complete_Deployment/img_14.png)

This mean that the gNB and the UE working in the other 2 terminals are connected to the CN5G.

To leave this visualization press Ctrl + c.

**Step 3:** Stoping the system simulation

To stop the simulation, use the command below in the CN5G terminal:
```
docker compose -f docker-compose-basic-nrf.yaml down
```
And close the other 2 terminals.

## Sources
[CN5G](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/develop/doc/NR_SA_Tutorial_OAI_CN5G.md)

[OAI gNB and nrUE](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/develop/doc/NR_SA_Tutorial_OAI_nrUE.md)